# Learning Regressions with Maximum Likelihood Estimation (MLE)

In [1]:
import time

import numpy as np
import pandas as pd

In [2]:
N = 200
beta = np.array([3.0, -5.0, 1.0])
p = len(beta)
beta = beta.reshape((p, 1))
sigma_squared = 10.0
theta_truth = tuple(float(b) for b in beta) + (sigma_squared,)

In [3]:
columns = tuple(f"β[{p_}]" for p_ in range(p)) + ("σ²",)
results = pd.DataFrame(columns=columns)
results.loc['Truth'] = theta_truth
results

,β[0],β[1],β[2],σ²
Truth,3.0,-5.0,1.0,10.0


## Make fake data

In [4]:
class FakeData(object):
    def __init__(self, N, p, beta, sigma_squared, rng=None):
        if not rng: 
            rng = np.random.RandomState()
        X = rng.uniform(size=N * p).reshape((N, p))
        error = rng.normal(loc=0, scale=sigma_squared ** 0.5, size=N).reshape((N, 1))
        X_times_beta = X @ beta
        assert X_times_beta.shape == (N, 1), X_times_beta.shape
        y = X_times_beta + error
        assert y.shape == (N, 1), y.shape
        self.X = X
        self.y = y
        return


In [5]:
# make y, X
rng = np.random.RandomState(seed=0)
fake_data = FakeData(N, p, beta, sigma_squared, rng=rng)
y, X = fake_data.y, fake_data.X
y.shape, X.shape

((200, 1), (200, 3))

## Least Squares

In [6]:
beta_hat_ols = np.linalg.inv(X.T @ X) @ X.T @ y
beta_hat_ols = np.linalg.inv(X.T @ X) @ X.T @ y
residual_ols = y - X @ beta_hat_ols
rss_ols = residual_ols.T @ residual_ols
sigma_squared_hat_ols = rss_ols[0, 0] / (N - p)
# sigma_hat_ols = sigma_squared_hat_ols ** 0.5
theta_ols = tuple(float(b) for b in beta_hat_ols) + (sigma_squared_hat_ols,)

In [7]:
results.loc['least squares'] = theta_ols

In [8]:
print("====Results so far====")
results

====Results so far====


,β[0],β[1],β[2],σ²
Truth,3.000000,-5.000000,1.000000,10.000000
least squares,3.865276,-6.848503,1.257617,10.015444


## MLE: likelihood + scipy.optimize.fmin

### define likelihood

In [9]:
from scipy import optimize
from scipy.stats import norm
from scipy.stats import multivariate_normal

In [10]:
class Likelihood(object):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __call__(self, theta):
        N, p = self.X.shape
        if N > 250:
            print("Not evaluating. N is too big:", N)
            return 0
        sigma_squared = theta[-1]
        beta = np.array(theta[0:p]).reshape((p, 1))
        mu = self.X @ beta
        joint_probability = multivariate_normal.pdf(
            y[:, 0],
            mu[:, 0],
            sigma_squared)
        return joint_probability


In [11]:
class LoggedFunction(object):
    def __init__(self, f, every_s=1):
        self.f = f
        self.every_s = every_s
        self.n_calls = 0
        self.time_of_last_log = time.time()

    def __call__(self, *args, **kwargs):
        self.n_calls += 1
        start = time.time()
        if (start - self.time_of_last_log) > 5:
            return "giving up!"
        its_been_a_while = (start - self.time_of_last_log) > self.every_s
        lets_log = its_been_a_while or self.n_calls == 1
        if lets_log:
            print(f"Call #{self.n_calls}:")
            print(*args, **kwargs)
        result = self.f(*args, **kwargs)
        if lets_log:
            print(result)
            end = time.time()
            print("wall time (s):", end - start)
            self.time_of_last_log = start
        return result


In [12]:
likelihood = Likelihood(X, y)

### optimize likelihood

In [13]:
initial_guess = tuple(0.0 for _ in range(p)) + (1.0,)

In [14]:
# test out the function
likelihood(theta_truth)

9.733278699026264e-226

In [15]:
negative_likelihood = lambda x: -1 * likelihood(x)
# f = negative_likelihood
f = LoggedFunction(negative_likelihood, every_s=1)
theta_mle_likelihood = optimize.fmin(f, theta_ols, xtol=1e-6, ftol=1e-6, maxiter=1000)

Call #1:
[ 3.8652763  -6.84850302  1.25761686 10.01544421]
-2.172626528930063e-223
wall time (s): 0.010221004486083984
Call #121:
[ 3.8654289  -6.84847264  1.2575046   9.86492828]
-2.1974564232158585e-223
wall time (s): 0.008758783340454102
Optimization terminated successfully.
         Current function value: -0.000000
         Iterations: 111
         Function evaluations: 200


In [16]:
results.loc['MLE w/ likelihood'] = theta_mle_likelihood

In [18]:
print("====Results so far====")
results

====Results so far====


,β[0],β[1],β[2],σ²
Truth,3.000000,-5.000000,1.000000,10.000000
least squares,3.865276,-6.848503,1.257617,10.015444
MLE w/ likelihood,3.865277,-6.848503,1.257617,9.865213


## MLE: negative log likelihood + scipy.optimize.fmin

### define negative log likelihood

In [19]:
class NegativeLogLikelihood(object):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __call__(self, theta):
        N, p = self.X.shape
        sigma_squared = theta[-1]
        beta = np.array(theta[0:p]).reshape((p, 1))
        log_probabilities = norm.logpdf(
            x=self.y[:, 0],
            loc=(self.X @ beta)[:, 0],
            scale=sigma_squared ** 0.5
        )
        negative_log_likelihood = -1 * sum(log_probabilities)
        return negative_log_likelihood

negative_log_likelihood = NegativeLogLikelihood(X, y)

In [20]:
initial_guess = tuple(0.0 for _ in range(p)) + (1.0,)

In [21]:
negative_log_likelihood(initial_guess)

1842.5164426644474

In [22]:
negative_log_likelihood(theta_truth)

518.1086802091776

In [23]:
negative_log_likelihood(theta_ols)

512.7005389200459

In [24]:
negative_log_likelihood(theta_mle_likelihood)

512.6891751390416

### optimize negative log likelihood

In [25]:
negative_log_likelihood(theta_ols)

512.7005389200459

In [26]:
f = LoggedFunction(negative_log_likelihood, every_s=1)
theta_mle_negloglikelihood = optimize.fmin(f, initial_guess, ftol=1e-6, xtol=1e-6, maxiter=3000)
print("")
print("solution:")
print(theta_mle_negloglikelihood)
print(f(theta_mle_negloglikelihood))

Call #1:
[0. 0. 0. 1.]
1842.5164426644474
wall time (s): 0.0008199214935302734
Optimization terminated successfully.
         Current function value: 512.689175
         Iterations: 518
         Function evaluations: 878

solution:
[ 3.86527621 -6.84850285  1.25761718  9.86521207]
512.6891751390419


In [27]:
results.loc['MLE w/ negative log likelihood'] = theta_mle_negloglikelihood

In [28]:
print("====Results so far====")
print(results)
results

====Results so far====
                                    β[0]      β[1]      β[2]         σ²
Truth                           3.000000 -5.000000  1.000000  10.000000
least squares                   3.865276 -6.848503  1.257617  10.015444
MLE w/ likelihood               3.865277 -6.848503  1.257617   9.865213
MLE w/ negative log likelihood  3.865276 -6.848503  1.257617   9.865212


,β[0],β[1],β[2],σ²
Truth,3.000000,-5.000000,1.000000,10.000000
least squares,3.865276,-6.848503,1.257617,10.015444
MLE w/ likelihood,3.865277,-6.848503,1.257617,9.865213
MLE w/ negative log likelihood,3.865276,-6.848503,1.257617,9.865212


### Past results

```
ftol=1e-11, xtol=1e-11, maxiter=1000
[ 3.01147067 -5.01263143  0.99964318  1.00115215]
142009.0002800877

ftol=1e-14, xtol=1e-14, maxiter=1000

Optimization terminated successfully.
         Current function value: 142009.000280
         Iterations: 661
         Function evaluations: 1191

solution:
[ 3.01147067 -5.01263143  0.99964318  1.00115215]
142009.0002800877


N = 1000000
ftol=1e-14, xtol=1e-14, maxiter=1000
... took too long!
ftol=1e-6, xtol=1e-6
Call #796:
[ 2.99618089 -5.00124526  1.00226668  0.99884177]
1417772.6734368044
wall time (s): 0.24301505088806152
Warning: Maximum number of function evaluations has been exceeded.

solution:
[ 2.99622187 -5.0012157   1.00224932  0.99883615]
```

- Issue: This fails for `N = 100000` and without `xtol`. 
  - Solution: go for more iterations
  - Solution: specify `xtol=1e-7`, everything is fine. Not sure why.
  - Source: "when you set ftol smaller than xtol during the search, xtol is exceeded before ftol is reached so it terminates prematurely" (https://stackoverflow.com/questions/9667514/what-is-the-difference-between-xtol-and-ftol-to-use-fmin-of-scipy-optimize#comment12285824_9669373)
- Issue: Fails when sigma_squared is large, e.g. 2500 (50 squared)

```
N = 100000
beta = np.array([3.0, -5.0, 1.0])
p = len(beta)
beta = beta.reshape((p, 1))
sigma = 10.0
                                    β[0]      β[1]      β[2]          σ
Truth                           3.000000 -5.000000  1.000000  10.000000
least squares                   3.114704 -5.126315  0.996433  10.011672
MLE w/ likelihood               0.000000  0.000000  0.000000   1.000000
MLE w/ negative log likelihood  3.114709 -5.126314  0.996428  10.011522

N = 100000
beta = np.array([3.0, -5.0, 1.0])
p = len(beta)
beta = beta.reshape((p, 1))
sigma = 50.0
                                    β[0]      β[1]      β[2]          σ
Truth                           3.000000 -5.000000  1.000000  50.000000
least squares                   3.573520 -5.631574  0.982167  50.058358
MLE w/ likelihood               0.000000  0.000000  0.000000   1.000000
MLE w/ negative log likelihood  3.753647 -3.448924 -1.755966  50.068533
```
- Issue: fails with a large coefficient

```
N = 100000
beta = np.array([3.0, -5.0, 1.0])
p = len(beta)
beta = beta.reshape((p, 1))
sigma = 10.0
                                    β[0]      β[1]      β[2]          σ
Truth                           3.000000 -5.000000  1.000000  10.000000
least squares                   3.114704 -5.126315  0.996433  10.011672
MLE w/ likelihood               0.000000  0.000000  0.000000   1.000000
MLE w/ negative log likelihood  3.114709 -5.126314  0.996428  10.011522
```

```
N = 100000
beta = np.array([3.0, -5.0, 1.0])
p = len(beta)
beta = beta.reshape((p, 1))
sigma_squared = 100.0

                                    β[0]      β[1]      β[2]          σ²
Truth                           3.000000 -5.000000  1.000000  100.000000
least squares                   3.114704 -5.126315  0.996433  100.233566
MLE w/ likelihood               0.000000  0.000000  0.000000    1.000000
MLE w/ negative log likelihood  3.114705 -5.126320  0.996436  100.230576

```

- another run...

```
N = 1000000
beta = np.array([3.0, -5.0, 1.0])
p = len(beta)
beta = beta.reshape((p, 1))
sigma_squared = 1.0

Optimization terminated successfully.
         Current function value: 1417772.672752
         Iterations: 449
         Function evaluations: 746

solution:
[ 2.99614446 -5.00118483  1.00229821  0.9976711 ]
1417772.6727520851

                                    β[0]      β[1]      β[2]        σ²
Truth                           3.000000 -5.000000  1.000000  1.000000
least squares                   2.996144 -5.001184  1.002298  0.997674
MLE w/ likelihood               0.000000  0.000000  0.000000  1.000000
MLE w/ negative log likelihood  2.996144 -5.001185  1.002298  0.997671
```


## MLE: negative log likelihood + scipy.optimize.newton

In [29]:
class NegativeLogLikelihood(object):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __call__(self, theta):
        N, p = self.X.shape
        sigma_squared = theta[-1]
        beta = np.array(theta[0:p]).reshape((p, 1))
        log_probabilities = norm.logpdf(
            x=self.y[:, 0],
            loc=(self.X @ beta)[:, 0],
            scale=sigma_squared ** 0.5
        )
        negative_log_likelihood = -1 * sum(log_probabilities)
        return negative_log_likelihood
    
    def gradient_wrt_theta(self, theta):
        N, p = self.X.shape
        sigma_squared = theta[-1]
        beta = np.array(theta[0:p]).reshape((p, 1))
        beta_portion = self.X.T @ self.y - self.X.T @ self.X @ beta
        residuals = y - X @ beta
        sigma_squared_portion = -0.5 / sigma_squared * (1 - ((residuals.T @ residuals)[0, 0] / sigma_squared))
        sigma_squared_portion = np.array(sigma_squared_portion).reshape((1, 1))
        gradient_log_likelihood = np.append(beta_portion, sigma_squared_portion)
        gradient_neg_log_likelihood = -1 * gradient_log_likelihood
        return gradient_neg_log_likelihood


negative_log_likelihood = NegativeLogLikelihood(X, y)

In [30]:
print(theta_ols)
negative_log_likelihood(theta_ols)

(3.8652763030229442, -6.848503020860781, 1.2576168609786866, 10.015444211889589)


512.7005389200459

In [31]:
negative_log_likelihood.gradient_wrt_theta((2.9687765223304448, -5.003006688218203, 1.005027854371277458, 1.0036908371459068))

array([   21.79907027,    73.00804163,    33.66085906, -1031.73484847])

# Appendix: numpy & scipy functions

In [32]:
# make y, X
rng = np.random.RandomState(seed=0)
fake_data = FakeData(10, p, beta, sigma_squared, rng=rng)
y, X = fake_data.y, fake_data.X
y.shape, X.shape

((10, 1), (10, 3))

In [33]:
norm.pdf(
    x=y[:, 0],
    loc=(X @ beta)[:, 0],
    scale=sigma_squared ** 0.5
)

array([0.0868254 , 0.09578631, 0.00959873, 0.04381317, 0.12602462,
       0.12396575, 0.03897101, 0.04286333, 0.12465125, 0.11745094])

In [34]:
norm.pdf(
    x=y[:, 0],
    loc=(X @ beta)[:, 0],
    scale=sigma_squared ** 0.5
)

array([0.0868254 , 0.09578631, 0.00959873, 0.04381317, 0.12602462,
       0.12396575, 0.03897101, 0.04286333, 0.12465125, 0.11745094])

In [35]:
np.prod(norm.pdf(
    x=y[:, 0],
    loc=(X @ beta)[:, 0],
    scale=sigma_squared ** 0.5
))

1.3363083265147978e-12

In [36]:
multivariate_normal.pdf(
    x=y[:, 0],
    mean=(X @ beta)[:, 0],
    cov=sigma_squared
)

1.3363083265147937e-12

In [37]:
np.log(norm.pdf(
    x=y[:, 0],
    loc=(X @ beta)[:, 0],
    scale=sigma_squared ** 0.5
))

array([-2.44385605, -2.34563554, -4.64612411, -3.12782084, -2.071278  ,
       -2.08774998, -3.24493714, -3.14973868, -2.08223543, -2.14173453])

In [38]:
norm.logpdf(
    x=y[:, 0],
    loc=(X @ beta)[:, 0],
    scale=sigma_squared ** 0.5
)

array([-2.44385605, -2.34563554, -4.64612411, -3.12782084, -2.071278  ,
       -2.08774998, -3.24493714, -3.14973868, -2.08223543, -2.14173453])

In [39]:
-1 * norm.logpdf(
    x=y[:, 0],
    loc=(X @ beta)[:, 0],
    scale=sigma_squared ** 0.5
)

array([2.44385605, 2.34563554, 4.64612411, 3.12782084, 2.071278  ,
       2.08774998, 3.24493714, 3.14973868, 2.08223543, 2.14173453])

In [40]:
# log likelihood of the truth!
-1 * sum(norm.logpdf(
    x=y[:, 0],
    loc=(X @ beta)[:, 0],
    scale=sigma_squared ** 0.5
))

27.341110284121008

# Other stuff

so...
- where is likelihood highest?
- where is L' = 0 and L'' < 0?
- let l = log(L)
- where is l' = 0 and l'' < 0
- where is -l' = 0 and -l'' > 0, in parameter space

fisher scoring... using scoring function l'. then use newton raphson to find root. 


In [41]:
beta = 
likelihood = lambda X, y, beta, sigma: np.prod(rng.normal(loc=X[i]@beta, scale=sigma))
negative_log_likelihood = lambda X, y: -1 * np.log(likelihood(X, y))
scoring_function = negative_log_likelihood.differentiate(wrt=beta)

while scoring_function(y, X) < epsilon:
    scoring_function_derivative = scoring_function.differentiate()
    gradient_vector = scoring_function_derivative(y, X)
    beta_update = -1 * step_size * gradient_vector
    beta += beta_update

SyntaxError: invalid syntax (<ipython-input-41-5a490a51cf19>, line 1)

In [ ]:
class Parameter(object):
    def __init__(self, shape):
        self.value = np.ones(shape=shape, dtype=np.float)

class LinearModel(object):
    def __init__(self, beta):
        self.beta = beta

    def __call__(self, X):
        return X @ beta

    def differentiate(self):
        return self.beta

class Product(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def differentiate(self, wrt=):
        return self.x
        

In [ ]:
beta_estimate = Parameter(shape=(p,))
linear_model = LinearModel(beta_estimate)  # takes an argument X
sigma_squared_estimate = Parameter(shape=(1,))
p_y = GaussianDistribution(mean=linear_model, variance=sigma_squared_estimate)
likelihood = Product(p_y(y[i], X[i]) for i in range(N))

In [ ]:
while True:
    negative_log_likelihood

# OLS with linear algebra

In [ ]:
# compute OLS fit
beta_hat_ols = np.linalg.inv(X.T @ X) @ X.T @ y
residual_ols = y - X @ beta_hat_ols
rss_ols = residual_ols.T @ residual_ols
sigma_squared_hat_ols = rss_ols / (N - p)
var_beta_ols = np.linalg.inv(X.T @ X) * sigma_squared_hat_ols

In [ ]:
beta_hat_ols

In [ ]:
# double check with scikit-learn
from sklearn.linear_model import LinearRegression
linear_regression = LinearRegression(fit_intercept=False)
linear_regression.fit(X, y)
linear_regression.coef_

In [ ]:
sigma_squared_hat_ols ** 0.5

In [ ]:
var_beta_ols